In [2]:
from PIL import Image
import numpy as np
import random

In [4]:
def divide_image_into_patches(image_path, patch_size=512):
    """
    Büyük bir görüntüyü belirtilen boyutta yamalara böler.
    
    Args:
        image_path (str): Görüntü dosyasının yolu.
        patch_size (int): Her yamanın piksel boyutu (varsayılan 512).
    
    Returns:
        list: Her biri numpy dizisi olan yama listesi.
    """
    # Görüntüyü aç ve numpy dizisine çevir
    image = Image.open(image_path).convert("RGB")
    image_array = np.array(image)
    height, width, _ = image_array.shape
    
    patches = []
    for i in range(0, height, patch_size):
        for j in range(0, width, patch_size):
            # Yamayı kes
            patch = image_array[i:min(i + patch_size, height), j:min(j + patch_size, width)]
            # Eğer yama tam boyutta değilse, doldur
            if patch.shape[0] != patch_size or patch.shape[1] != patch_size:
                padded_patch = np.zeros((patch_size, patch_size, 3), dtype=np.uint8)
                padded_patch[:patch.shape[0], :patch.shape[1]] = patch
                patch = padded_patch
            patches.append(patch)
    return patches

# Örnek kullanım
image_path = "app.jpeg"  # Kendi görüntünüzün yolunu buraya ekleyin
patches = divide_image_into_patches(image_path)
print(f"Toplam {len(patches)} yama oluşturuldu.")

Toplam 4 yama oluşturuldu.


In [6]:
patches

[array([[[109, 133, 143],
         [106, 130, 140],
         [107, 131, 141],
         ...,
         [129, 154, 159],
         [128, 153, 158],
         [128, 153, 158]],
 
        [[109, 133, 143],
         [106, 130, 140],
         [107, 131, 141],
         ...,
         [129, 154, 159],
         [128, 153, 158],
         [128, 153, 158]],
 
        [[109, 133, 143],
         [107, 131, 141],
         [107, 131, 141],
         ...,
         [129, 154, 159],
         [128, 153, 158],
         [128, 153, 158]],
 
        ...,
 
        [[ 75,  80,  83],
         [ 74,  79,  82],
         [ 74,  79,  82],
         ...,
         [ 65,  62,  47],
         [ 64,  61,  46],
         [ 63,  60,  45]],
 
        [[ 65,  70,  73],
         [ 65,  70,  73],
         [ 64,  69,  72],
         ...,
         [ 61,  58,  41],
         [ 60,  57,  40],
         [ 60,  57,  40]],
 
        [[ 54,  59,  62],
         [ 54,  59,  62],
         [ 53,  58,  61],
         ...,
         [ 61,  58,  41],
  

In [8]:
def encode_patch_to_tokens(patch, num_tokens=64):
    """
    Bir yamayı belirtilen sayıda tokena sıkıştırır.
    
    Args:
        patch (np.array): 512x512x3 boyutunda yama.
        num_tokens (int): Üretilecek token sayısı (varsayılan 64).
    
    Returns:
        list: Her biri bir token olan liste (simüle edilmiş sayılar).
    """
    # Yamayı düzleştir (262,144 x 3)
    flattened = patch.reshape(-1, 3)
    pixels_per_token = flattened.shape[0] // num_tokens  # 4096 piksel/token
    
    tokens = []
    for i in range(num_tokens):
        start = i * pixels_per_token
        end = start + pixels_per_token
        token_pixels = flattened[start:end]
        # Tokenı temsil etmek için ortalama RGB değeri
        token = np.mean(token_pixels, axis=0).tolist()
        tokens.append(token)
    return tokens

# Tüm yamaları tokenlara çevir
patch_tokens = [encode_patch_to_tokens(patch) for patch in patches]
print(f"Her yama {len(patch_tokens[0])} tokena kodlandı.")

Her yama 64 tokena kodlandı.


In [10]:
patch_tokens

[[[107.730712890625, 126.15673828125, 130.179443359375],
  [109.221923828125, 127.397216796875, 131.6240234375],
  [111.3291015625, 129.4921875, 133.479736328125],
  [113.89697265625, 131.5595703125, 135.259521484375],
  [113.03955078125, 130.85205078125, 134.44873046875],
  [115.454345703125, 133.654296875, 137.001953125],
  [119.689697265625, 137.594970703125, 140.5498046875],
  [122.297607421875, 137.378662109375, 140.153076171875],
  [125.992919921875, 135.9111328125, 138.4423828125],
  [133.64501953125, 136.93115234375, 138.421142578125],
  [144.038818359375, 138.53076171875, 138.54736328125],
  [150.18017578125, 137.39404296875, 136.306640625],
  [155.70751953125, 135.4462890625, 133.658203125],
  [149.91064453125, 124.12353515625, 122.6328125],
  [149.3583984375, 124.04150390625, 122.4150390625],
  [149.345947265625, 130.420654296875, 128.434326171875],
  [150.46044921875, 137.738525390625, 134.950439453125],
  [153.12255859375, 140.675048828125, 138.04052734375],
  [154.4060058

In [12]:
def combine_tokens_with_separator(patch_tokens, separator_token="<patch_sep>"):
    """
    Yama tokenlarını özel bir ayırıcıyla birleştirir.
    
    Args:
        patch_tokens (list): Her yamanın token listesi.
        separator_token (str): Yamaları ayırmak için kullanılacak token.
    
    Returns:
        list: Birleştirilmiş token dizisi.
    """
    combined = []
    for i, tokens in enumerate(patch_tokens):
        combined.extend(tokens)
        if i < len(patch_tokens) - 1:  # Son yamada ayırıcı ekleme
            combined.append(separator_token)
    return combined

# Tokenları birleştir
combined_tokens = combine_tokens_with_separator(patch_tokens)
print(f"Birleştirilmiş token sayısı: {len(combined_tokens)}")

Birleştirilmiş token sayısı: 259


In [14]:
def generate_description_from_tokens(combined_tokens):
    """
    Tokenlardan basit bir metin açıklaması üretir.
    
    Args:
        combined_tokens (list): Birleştirilmiş token dizisi.
    
    Returns:
        str: Üretilen açıklama.
    """
    description = []
    patch_idx = 0
    for token in combined_tokens:
        if token == "<patch_sep>":
            patch_idx += 1
            description.append(f"\nYama {patch_idx}:")
        elif isinstance(token, list):  # RGB token
            r, g, b = token
            color_desc = f"Ortalama renk RGB({int(r)}, {int(g)}, {int(b)})"
            if not description or description[-1].startswith("\n"):
                description.append(f" {color_desc}")
            else:
                description[-1] += f", {color_desc}"
    return "".join(description)

# Açıklama üret
response = generate_description_from_tokens(combined_tokens)
print("Üretilen Açıklama:")
print(response)

Üretilen Açıklama:
 Ortalama renk RGB(107, 126, 130), Ortalama renk RGB(109, 127, 131), Ortalama renk RGB(111, 129, 133), Ortalama renk RGB(113, 131, 135), Ortalama renk RGB(113, 130, 134), Ortalama renk RGB(115, 133, 137), Ortalama renk RGB(119, 137, 140), Ortalama renk RGB(122, 137, 140), Ortalama renk RGB(125, 135, 138), Ortalama renk RGB(133, 136, 138), Ortalama renk RGB(144, 138, 138), Ortalama renk RGB(150, 137, 136), Ortalama renk RGB(155, 135, 133), Ortalama renk RGB(149, 124, 122), Ortalama renk RGB(149, 124, 122), Ortalama renk RGB(149, 130, 128), Ortalama renk RGB(150, 137, 134), Ortalama renk RGB(153, 140, 138), Ortalama renk RGB(154, 136, 134), Ortalama renk RGB(154, 132, 129), Ortalama renk RGB(153, 129, 127), Ortalama renk RGB(152, 128, 126), Ortalama renk RGB(152, 130, 128), Ortalama renk RGB(147, 130, 128), Ortalama renk RGB(138, 121, 120), Ortalama renk RGB(134, 117, 116), Ortalama renk RGB(133, 117, 116), Ortalama renk RGB(132, 119, 117), Ortalama renk RGB(122, 102, 

In [34]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
from transformers import SiglipVisionModel, SiglipProcessor, AutoTokenizer, LlamaForCausalLM

# 1. Görüntüyü 512x512 yamalarına bölme
def divide_image_into_patches(image_path, patch_size=512):
    image = Image.open(image_path).convert("RGB")
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image_tensor = preprocess(image)
    height, width = image_tensor.shape[1], image_tensor.shape[2]
    patches = []
    for i in range(0, height, patch_size):
        for j in range(0, width, patch_size):
            patch = image_tensor[:, i:min(i + patch_size, height), j:min(j + patch_size, width)]
            if patch.shape[1] != patch_size or patch.shape[2] != patch_size:
                padded_patch = torch.zeros(3, patch_size, patch_size)
                padded_patch[:, :patch.shape[1], :patch.shape[2]] = patch
                patch = padded_patch
            patches.append(patch)
    return torch.stack(patches)

# 2. SmolVLM benzeri model (SigLIP + Llama)
class SmolVLM(nn.Module):
    def __init__(self, vision_model_name="google/siglip-base-patch16-224", language_model_name="meta-llama/Llama-2-7b-hf"):
        super(SmolVLM, self).__init__()
        # SigLIP vizyon modeli
        self.vision_model = SiglipVisionModel.from_pretrained(vision_model_name)
        self.processor = SiglipProcessor.from_pretrained(vision_model_name)
        
        # Llama dil modeli (örnek olarak, daha küçük bir model de seçilebilir)
        self.language_model = LlamaForCausalLM.from_pretrained(language_model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(language_model_name)
        
        # SigLIP çıkışını Llama girişine uyarlama için projeksiyon
        self.proj = nn.Linear(self.vision_model.config.hidden_size, self.language_model.config.hidden_size)
        
        # Özel tokenlar için ekleme (örneğin <patch_sep>)
        self.tokenizer.add_special_tokens({"additional_special_tokens": ["<patch_sep>"]})
        self.language_model.resize_token_embeddings(len(self.tokenizer))

    def forward(self, patches, text_input=None):
        # SigLIP ile vizyon tokenlarını çıkar
        batch_size, num_patches, c, h, w = patches.shape
        patches = patches.view(batch_size * num_patches, c, h, w)
        
        # SigLIP girişi için tensörleri PIL görüntüsüne çevir (processor için)
        patch_images = [transforms.ToPILImage()(patch) for patch in patches]
        inputs = self.processor(images=patch_images, return_tensors="pt")
        
        with torch.no_grad():
            vision_outputs = self.vision_model(**inputs)
        vision_tokens = vision_outputs.last_hidden_state  # [num_patches, seq_len, hidden_size]
        
        # Projeksiyon ile dil modeline uyarla
        vision_tokens = self.proj(vision_tokens)  # [num_patches, seq_len, llama_hidden_size]
        
        # Yamaları birleştir ve <patch_sep> ekle
        sep_token_id = self.tokenizer.convert_tokens_to_ids("<patch_sep>")
        combined_tokens = []
        for i in range(num_patches):
            combined_tokens.append(vision_tokens[i])
            if i < num_patches - 1:
                combined_tokens.append(torch.full((1, 1, vision_tokens.size(-1)), sep_token_id, dtype=torch.float32))
        vision_tokens = torch.cat(combined_tokens, dim=0)  # [total_seq_len, llama_hidden_size]

        # Dil modeli ile metin üret
        if text_input is not None:
            text_inputs = self.tokenizer(text_input, return_tensors="pt", padding=True)
            input_ids = text_inputs["input_ids"]
            embeddings = self.language_model.model.embed_tokens(input_ids)
            combined_inputs = torch.cat([vision_tokens.unsqueeze(0), embeddings], dim=1)
            outputs = self.language_model(inputs_embeds=combined_inputs)
            return outputs.logits
        return vision_tokens

    def generate(self, patches, max_len=50):
        vision_tokens = self.forward(patches)  # [total_seq_len, hidden_size]
        batch_size = 1  # Tek görüntü için
        
        # Başlangıç tokenı ile metin üretimi
        input_ids = torch.tensor([[self.tokenizer.bos_token_id]], dtype=torch.long)
        for _ in range(max_len):
            embeddings = self.language_model.model.embed_tokens(input_ids)
            combined_inputs = torch.cat([vision_tokens.unsqueeze(0), embeddings], dim=1)
            with torch.no_grad():
                logits = self.language_model(inputs_embeds=combined_inputs).logits
            next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
            input_ids = torch.cat([input_ids, next_token], dim=-1)
            if next_token.item() == self.tokenizer.eos_token_id:
                break
        
        return input_ids

# 3. İş akışı
def process_image_to_text(image_path, model, max_len=20):
    patches = divide_image_into_patches(image_path)
    print(f"Toplam {patches.shape[0]} yama oluşturuldu.")
    
    patches = patches.unsqueeze(0)  # [1, num_patches, 3, 512, 512]
    output_ids = model.generate(patches, max_len=max_len)
    generated_text = model.tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text

# Test
if __name__ == "__main__":
    # Modeli oluştur
    model = SmolVLM()
    
    # Test görüntüsü
    image_path = "app.jpeg"  # Görüntü yolunu güncelleyin
    generated_text = process_image_to_text(image_path, model)
    print("Üretilen Metin:")
    print(generated_text)

config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]